In [1]:
max_episodes = 0 # Set to 0 to use all episodes



import os
import dataiku

from dataiku import spark as dkuspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from dataiku.customrecipe import *
#from deepspeech import Model
import scipy.io.wavfile as wav
import speech_recognition as sr


from pyspark.sql.functions import udf, col
from pyspark.sql.types import *



/usr/hdp/current/spark2-client/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py:1890: DeprecationWarning: invalid escape sequence \*
/usr/hdp/current/spark2-client/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py:1890: DeprecationWarning: invalid escape sequence \*
/usr/hdp/current/spark2-client/python/pyspark/sql/readwriter.py:398: DeprecationWarning: invalid escape sequence \`
  """
/usr/hdp/current/spark2-client/python/pyspark/sql/readwriter.py:759: DeprecationWarning: invalid escape sequence \`
  """
/usr/hdp/current/spark2-client/python/pyspark/sql/streaming.py:618: DeprecationWarning: invalid escape sequence \`
  """
/usr/hdp/current/spark2-client/python/pyspark/sql/functions.py:1519: DeprecationWarning: invalid escape sequence \d
  """


Exception: Java gateway process exited before sending the driver its port number

In [ ]:

# Read recipe inputs

audios = dataiku.Folder("episode_wavs_hdfs")
#episode_wavs_hdfs_info = episode_wavs_hdfs.get_info()

idx = 0

path_list = []
for path in audios.list_paths_in_partition():
    path_list.append(path)
    idx += 1
    if idx == max_episodes:
        break

In [ ]:

cSchema = StructType([StructField("path", StringType())])


df = spark.createDataFrame(path_list,schema=cSchema)

In [ ]:

def read_audio(path):
    r = sr.Recognizer()

    with audios.get_download_stream(path[1:]) as stream:
        with sr.AudioFile(stream) as source:
            audio_google = r.record(source)
    try:
        text = r.recognize_google(audio_google)
    except:
        text = "NA"
    return text


read_audio_udf = udf(read_audio, StringType())


In [ ]:

df.withColumn("text",read_audio_udf(col("path")))



In [ ]:

 # Write recipe outputs
detected_texts = dataiku.Dataset("episodes_text_spark")
detected_texts.write_with_schema(detected_texts, df)

In [1]:

import os
import dataiku

from dataiku import spark as dkuspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

/data/dataiku/dataiku-dss-8.0.1/spark-standalone-home/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py:2020: DeprecationWarning: invalid escape sequence \*
/data/dataiku/dataiku-dss-8.0.1/spark-standalone-home/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py:2020: DeprecationWarning: invalid escape sequence \*


In [7]:
f = dataiku.Folder("temp_episodes")
with f.get_download_stream("image001.jpg") as file:
    my_file = file
f.upload_stream("upload.jpg", my_file)